In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn

In [2]:
X, y = load_boston(return_X_y=True)
y = y[..., np.newaxis]
y = np.concatenate((y, y * 2), axis=-1)
X = StandardScaler().fit_transform(X)

In [3]:
# X = X[:2]
# y = y[:2

In [4]:
w1 = torch.ones(13, 2)
w2 = torch.ones(2, 2)
b1 = torch.zeros(2)
b2 = torch.zeros(2)
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

In [5]:
for i in range(1):
    
    w1.requires_grad = True
    b1.requires_grad = True
    w2.requires_grad = True
    b2.requires_grad = True
    
    a1 = X_t @ w1 + b1
#     print(a1.shape)
#     print((a1 @ w2).shape)
    y__t = a1 @ w2 + b2
    
#     y__t.requires_grad = True
#     print(y__t.shape, y_t.shape)
    
    assert y__t.shape == y_t.shape
        
    l = 0.5 * torch.sum((y_t - y__t) ** 2) / len(y_t)
    l.backward()
#     print(w2.grad)
#     print(b2.grad)
#     print(w1.grad)
#     print(b1.grad)
    if i % 1 == 0:
        print(l)
    w1 = w1.data - 0.01 * w1.grad
    w2 = w2.data - 0.01 * w2.grad
    
    b1 = b1.data - 0.01 * b1.grad
    b2 = b2.data - 0.01 * b2.grad


    
    

tensor(1667.3312, grad_fn=<DivBackward0>)


In [6]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.ones([in_dim, out_dim])
#         self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([1, out_dim])
        self.dw = None
        self.db = None
        self.in_dim = self.w.shape[0]
        self.out_dim = self.w.shape[1]

        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        print(d.shape)
        
        self.db = np.mean(d, axis=0)
        assert self.db.shape == self.b.shape, (d.shape, self.db.shape, self.b.shape)
        
        J = np.zeros([self.x.shape[0], self.out_dim, np.prod(self.w.shape)])
        j = 0
        for i in range(self.out_dim):
            J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
            j += self.in_dim
        
        dw = d @ J
        
        dw = np.reshape(np.mean(dw, axis=0), self.w.shape, order='F')
        
        self.dw = dw
        
        d = d @ np.repeat(self.w.T[np.newaxis, ...], d.shape[0], axis=0)
        
#         print(self.dw, self.db)
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [7]:
class MeanSquaredError:
    def __init__(self):
        self.y_ = None
    def forward(self, y_, y):
        assert y_.shape == y.shape, (y_.shape, y.shape)
        self.y_ = y_
        self.y = y
#         print(y_)
        l = 0.5 * np.sum(np.square(y - y_)) / len(y)
        return l
    
    def backward(self):
        d = -(self.y - self.y_)[:, np.newaxis, ...]
        print(d.shape)
        return d

In [8]:
class LinearRegression:
    def __init__(self):
        self.linear1 = Linear(13, 2)
        self.linear2 = Linear(2, 2)
        self.loss = MeanSquaredError()
        
    def forward(self, x, y):
        x = self.linear1.forward(x)
        x = self.linear2.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self):
        d = self.loss.backward()
        d = self.linear2.backward(d)
        d = self.linear1.backward(d)
        
    
    def step(self, lr):
        self.linear1.step(lr)
        self.linear2.step(lr)
        

In [9]:
# X = np.array([[1, 2, 3], [1, 2, 3]])
# y = np.array([[4], [4]])

In [10]:
linreg = LinearRegression()

In [11]:
for i in range(1):
    loss = linreg.forward(X, y)
    if i % 1 == 0:
        print(loss, 'loss')
    linreg.backward()
    linreg.step(0.01)


1667.3309692990128 loss
(506, 1, 2)


RuntimeError: No active exception to reraise

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X, y)

In [ ]:
np.sum(0.5 * (lr.predict(X) - y) ** 2) / len(X)

In [ ]:
linreg.linear.forward(X).sum()

In [ ]:
lr.predict(X).sum()